# Imports

In [32]:
import pandas as pd
import numpy as np


# Raw Data

In [48]:
df = pd.read_csv('Part 1/hotel_booking.csv')
df = df.iloc[:, 1:32]
df.shape

(119390, 31)

In [54]:
# train test split
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.4, random_state=42)
test, validation = train_test_split(test, test_size=0.5, random_state=42)

# define the target variable
target = 'is_canceled'
features = df.columns.drop(target)

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target] 
X_validation = validation[features]
y_validation = validation[target]

# H20.ai

In [41]:
!pip install requests
!pip install tabulate
!pip install "colorama >= 0.3.8"
!pip install future
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o


Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html


In [45]:
pip install h2o-wave

Note: you may need to restart the kernel to use updated packages.


In [50]:
import h2o
from h2o.automl import H2OAutoML

In [51]:
# initialize the h2o session
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 min 02 secs
H2O_cluster_timezone:,America/Toronto
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 5 days
H2O_cluster_name:,H2O_from_python_chiaralu_bajvat
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.542 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [55]:
# transform df into a compatible data format
hf_dataset = h2o.H2OFrame(df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [56]:

hf_dataset.types

{'is_canceled': 'int',
 'lead_time': 'int',
 'arrival_date_year': 'int',
 'arrival_date_month': 'enum',
 'arrival_date_week_number': 'int',
 'arrival_date_day_of_month': 'int',
 'stays_in_weekend_nights': 'int',
 'stays_in_week_nights': 'int',
 'adults': 'int',
 'children': 'int',
 'babies': 'int',
 'meal': 'enum',
 'country': 'enum',
 'market_segment': 'enum',
 'distribution_channel': 'enum',
 'is_repeated_guest': 'int',
 'previous_cancellations': 'int',
 'previous_bookings_not_canceled': 'int',
 'reserved_room_type': 'enum',
 'assigned_room_type': 'enum',
 'booking_changes': 'int',
 'deposit_type': 'enum',
 'agent': 'int',
 'company': 'enum',
 'days_in_waiting_list': 'int',
 'customer_type': 'enum',
 'adr': 'real',
 'required_car_parking_spaces': 'int',
 'total_of_special_requests': 'int',
 'reservation_status': 'enum',
 'reservation_status_date': 'time'}

In [57]:
hf_dataset.describe()

Rows:119390
Cols:31

,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
type,int,int,int,enum,int,int,int,int,int,int,int,enum,enum,enum,enum,int,int,int,enum,enum,int,enum,int,enum,int,enum,real,int,int,enum,time
mins,0.0,0.0,2015.0,,1.0,1.0,0.0,0.0,0.0,0.0,0.0,,,,,0.0,0.0,0.0,,,0.0,,1.0,,0.0,,-6.38,0.0,0.0,,1413504000000.0
mean,0.37041628277075134,104.01141636652962,2016.1565541502669,,27.165172962559705,15.798241058715151,0.9275986263506132,2.500301532791692,1.856403383867997,0.10388990333874994,0.007948739425412514,,,,,0.03191222045397437,0.08711784906608591,0.13709690928888515,,,0.22112404724013748,,86.69338185346915,,2.321149174972779,,101.83112153446696,0.06251779881062071,0.5713627607002247,,1469838287883.411
maxs,1.0,737.0,2017.0,,53.0,31.0,19.0,50.0,55.0,10.0,10.0,,,,,1.0,26.0,72.0,,,21.0,,535.0,,391.0,,5400.0,8.0,5.0,,1505347200000.0
sigma,0.48291822659259837,106.86309704798794,0.7074759445196503,,13.605138355497608,8.780829470578363,0.9986134945978754,1.908285615047911,0.5792609988327545,0.39856144478644157,0.09743619130126435,,,,,0.17576714541065658,0.8443363841545108,1.4974368477076772,,,0.6523055726747717,,110.77454764295149,,17.59472087877622,,50.53579028554877,0.2452911474674938,0.7927984228094118,,19806197159.481823
zeros,75166,6345,0,,0,0,51998,7645,403,110796,118473,,,,,115580,112906,115770,,,101314,,0,,115692,,1959,111974,70318,,0
missing,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,16340,0,0,0,0,0,0,0,0
0,0.0,342.0,2015.0,July,27.0,1.0,0.0,0.0,2.0,0.0,0.0,BB,PRT,Direct,Direct,0.0,0.0,0.0,C,C,3.0,No Deposit,nan,nan,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00
1,0.0,737.0,2015.0,July,27.0,1.0,0.0,0.0,2.0,0.0,0.0,BB,PRT,Direct,Direct,0.0,0.0,0.0,C,C,4.0,No Deposit,nan,nan,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00
2,0.0,7.0,2015.0,July,27.0,1.0,0.0,1.0,1.0,0.0,0.0,BB,GBR,Direct,Direct,0.0,0.0,0.0,A,C,0.0,No Deposit,nan,nan,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00


In [59]:
train, test = hf_dataset.split_frame(ratios=[0.6],seed=1)
test, val = test.split_frame(ratios=[0.5],seed=1)

y = "is_canceled"
X = hf_dataset.columns
X.remove(y)

In [61]:
h2o_aml = H2OAutoML(max_models = 12, seed = 1, exclude_algos = ["StackedEnsemble"], verbosity="info") #max_runtime_secs=120,

In [63]:
h2o_aml.train(x = X, y = y, training_frame = train)

AutoML progress: |
00:13:09.806: Project: AutoML_1_20240419_01309
00:13:09.808: 5-fold cross-validation will be used.
00:13:09.809: Setting stopping tolerance adaptively based on the training frame: 0.0037349070401143008
00:13:09.809: Build control seed: 1
00:13:09.812: training frame: Frame key: AutoML_1_20240419_01309_training_py_8_sid_9207    cols: 31    rows: 71687  chunks: 5    size: 2545188  checksum: -36710179132475560
00:13:09.812: validation frame: NULL
00:13:09.812: leaderboard frame: NULL
00:13:09.812: blending frame: NULL
00:13:09.813: response column: is_canceled
00:13:09.813: fold column: null
00:13:09.813: weights column: null
00:13:09.838: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 90w), lr_search (7g, 30w)]}, {GLM : [def_1 (1g, 10w)]}, {DRF : [def_1 (2g, 10w), XRT (3g, 10w)]}, {GBM : [def_5 (1g, 10w), def_2 (2g, 10w), def_3 (2g, 10w), def_4 (2g, 10w), def_1 (3g, 10w), grid_1 (4g, 60w), lr_annealing (7g, 10w)]}, 

Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_2_AutoML_1_20240419_01309


Model Summary: 
    number_of_trees
--  -----------------
    71

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 6.403619813898192e-07
RMSE: 0.0008002262063878058
MAE: 0.0003632552862812014
RMSLE: 0.0006695756565994769
Mean Residual Deviance: 6.403619813898192e-07

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 7.661593317608457e-11
RMSE: 8.753052791802673e-06
MAE: 4.362590899120618e-06
RMSLE: 6.1959495609489645e-06
Mean Residual Deviance: 7.661593317608457e-11

Cross-Validation Metrics Summary: 
                        mean         sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  -----------  -----------  ------------  ------------  ------------  ------------  ------------
aic                     nan          0            nan           nan           nan           nan           nan
loglikelihood           nan          0            nan           nan           nan           nan           nan
mae                     4.36261e-06  8.20656e-07  3.60273e-06   3.5979e-06    5.2276e-06    4.16046e-06   5.22437e-06
mean_residual_deviance  7.66167e-11  2.63556e-11  5.24217e-11   4.68354e-11   9.222e-11     8.30275e-11   1.08579e-10
mse                     7.66167e-11  2.63556e-11  5.24217e-11   4.68354e-11   9.222e-11     8.30275e-11   1.08579e-10
r2                      1            0            1             1             1             1             1
residual_deviance       7.66167e-11  2.63556e-11  5.24217e-11   4.68354e-11   9.222e-11     8.30275e-11   1.08579e-10
rmse                    8.64382e-06  1.54153e-06  7.24028e-06   6.84364e-06   9.60312e-06   9.11194e-06   1.04201e-05
rmsle                   6.10687e-06  1.17064e-06  4.89432e-06   4.9652e-06    6.84142e-06   6.26221e-06   7.57119e-06

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------
    2024-04-19 00:16:58  6.768 sec   0                  0.5              0.5             0.25
    2024-04-19 00:16:58  6.995 sec   5                  0.0840442        0.0840442       0.00706343
    2024-04-19 00:16:59  7.522 sec   10                 0.0171632        0.0166665       0.000294577
    2024-04-19 00:17:00  8.228 sec   15                 0.00331435       0.00289751      1.09849e-05
    2024-04-19 00:17:00  8.947 sec   20                 0.00136645       0.000724399     1.86718e-06
    2024-04-19 00:17:01  9.579 sec   25                 0.00109476       0.000452752     1.19851e-06
    2024-04-19 00:17:02  10.200 sec  30                 0.0010375        0.000428198     1.0764e-06
    2024-04-19 00:17:02  10.813 sec  35                 0.000988814      0.000417076     9.77753e-07
    2024-04-19 00:17:03  11.411 sec  40                 0.000966514      0.000411632     9.3415e-07
    2024-04-19 00:17:03  12.016 sec  45                 0.000945141      0.000406536     8.93292e-07
    2024-04-19 00:17:04  12.656 sec  50                 0.000914948      0.000399285     8.3713e-07
    2024-04-19 00:17:05  13.352 sec  55                 0.000887245      0.000392085     7.87204e-07
    2024-04-19 00:17:05  14.011 sec  60                 0.000861456      0.000383121     7.42106e-07
    2024-04-19 00:17:06  14.732 sec  65                 0.000829113      0.000373644     6.87428e-07
    2024-04-19 00:17:07  15.439 sec  70                 0.000802872      0.00036357      6.44604e-07
    2024-04-19 00:17:07  15.645 sec  71                 0.000800226      0.000363255     6.40362e-07

Variable Importances: 
variable                      relative_importance     scaled_importance       percentage
----------------------------  ----------------------  ----------------------  ----------------------
reservation_st

# TPOT

# Amazon Lex